In [ ]:
# data access

import xarray as xr

In [ ]:
import display
import matplotlib.pyplot as plt

In [ ]:
# data access configuration

dataset = "s3://ecmwf-era5-single-levels/reanalysis-era5-single-levels.zarr"

In [ ]:
reference_period = slice("1981", "2010")
area = {"latitude": slice(55, 47), "longitude": slice(5, 16)}
location = {"latitude": 52.5, "longitude": 13.4}

In [ ]:
ds = xr.open_dataset(dataset, chunks={}, engine="zarr").astype("float32")
t2m = ds.t2m - 273.15
t2m.attrs["units"] = "C"

In [ ]:
%%time

t2m_area = t2m.sel(**area)
t2m_mean = t2m_area.sel(valid_time="2023-10").mean(dim="valid_time").compute()

In [ ]:
display.map(t2m_mean, vmax=None, cmap="Reds", title="Mean Surface Temperature, Oct 2023")

In [ ]:
%%time

t2m_area = t2m.sel(**area)
t2m_ref_mean = t2m_area.sel(valid_time=t2m_area["valid_time.month"] == 10).sel(valid_time=reference_period).mean(dim="valid_time")
t2m_anomaly = (t2m_area.sel(valid_time="2023-10").mean(dim="valid_time") - t2m_ref_mean).compute()
t2m_anomaly

In [ ]:
display.map(t2m_anomaly, vmax=None, cmap="Reds", title="Mean Surface Temperature anomaly (ref 1981-2010), Oct 2013")

In [ ]:
%%time

t2m_loc = t2m.sel(**location, method="nearest")
t2m_climatology_mean = t2m_loc.sel(valid_time=reference_period).groupby("valid_time.month").mean(dim="valid_time").compute()
t2m_climatology_std = t2m_loc.sel(valid_time=reference_period).groupby("valid_time.month").std(dim="valid_time").compute()
t2m_loc_monthly_means = t2m_loc.sel(valid_time="2023").resample(valid_time="1M").mean(dim="valid_time").compute()

In [ ]:
plt.figure(figsize=(10, 5))
t2m_climatology_mean.plot(label='Mean')
plt.errorbar(t2m_climatology_mean.month, t2m_climatology_mean, yerr=t2m_climatology_std, fmt='o', label='Standard Deviation')
for month in range (1, 11):
    t2m_point = t2m_loc_monthly_means.sel(valid_time=t2m_loc_monthly_means["valid_time.month"]==month)
    label = None
    if month == 1:
        label = "2023"
    plt.scatter(month, t2m_point, color='red', label=label)
plt.title('Surface Temperature climatology in Berlin (DE), 1981-2010')
plt.xticks(t2m_climatology_mean.month)
plt.xlabel('Month')
plt.ylabel('Surface Temperature [C]')
plt.legend()
plt.grid(alpha=0.3)
plt.show()